# __MODEL NOTEBOOK__

## *press these links to see the training notebooks*
[Model for detecting fall using CNN (Image classification)](https://www.kaggle.com/code/sahiltarlana2601/ai-sis-fall/edit)

[Model for detecting fall using Random forest with sensor data](https://www.kaggle.com/code/sahiltarlana2601/big-fall-pranav)


### __IMPORTING REQUIRED MODULES__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.patches as patches
from ultralytics import YOLO
import os
import cv2
import torch
from PIL import Image
import time
from twilio.rest import Client
import keys

### __VERIFYING VERSION OF TENSOR FLOW__

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.16.1


### __LOADING MODEL DOWNLOADED FROM KAGGLE__

In [3]:
from keras.models import load_model
model=load_model("pranav_sahil_final.h5")

### __LOADING YOLOV8 MODEL__

In [4]:
from ultralytics import YOLO

# Load the YOLOv8 model
yolo_model = YOLO("yolov8n.pt")

# Set the input size using the model's overrides parameter
yolo_model.model.args["imgsz"] = 640

In [5]:
def get_boxes_vid(frame):
    img = frame
    image_width, image_height, _ = img.shape
    results = yolo_model(img)
    myboxes = []
    for result in results:
        boxes =result.boxes
        for box in boxes:
            class_id=box.cls
            confidence=box.conf.item()
            x_min, y_min, x_max, y_max = box.xyxy[0]
            if class_id==0 and  confidence > 0.3:
                # print(f"Bounding box coordinates: ({x_min:.2f}, {y_min:.2f}), ({x_max:.2f}, {y_max:.2f})")
                temp = [x_min, y_min, x_max, y_max]
                myboxes.append(temp)
    return myboxes

In [6]:
# frame=cv2.imread("test_images/sample5.jpg")
# get_boxes_vid(frame)

In [7]:
pref_size = (128, 128)
def pred_vid(frame):
    list_fall=[]
    bounding_boxes=get_boxes_vid(frame)
    img=frame
    complete_images=[]
    if(bounding_boxes==None):
            return list_fall
    for box in bounding_boxes:
        image_height, image_width, _ = img.shape
        x_min, y_min, x_max, y_max = box[:]
        print(f"Bounding box coordinates: ({x_min:.2f}, {y_min:.2f}), ({x_max:.2f}, {y_max:.2f})")
        complete_images.append(img[int(y_min):int(y_max), int(x_min):int(x_max)])
    print(len(complete_images))
    for cropped_img in complete_images:
        print(1234)
        cropped_img_resized = cv2.resize(cropped_img, pref_size)
        # plt.imshow(cropped_img_resized)
        # plt.show()
        cropped_img_resized = cropped_img_resized / 255.0
        # print(cropped_img_resized.shape)
        cropped_img_resized = np.expand_dims(cropped_img_resized, axis=0)
        predictions = model.predict(cropped_img_resized)
        print(predictions)
        if(predictions[0][0]>=0.55):
            list_fall.append(1)
        else:
            list_fall.append(0)
    return list_fall

In [8]:
# frame=cv2.imread("test_images/sample5.jpg")
# plt.imshow(frame)
# plt.show()
# values=pred_vid(frame)
# if (len(values)==0):
#             print("no person detected")
# else:
#     for val in values:
#         if val == 1:
#             print("fall detected")
#             client=Client(keys.account_sid,keys.auth_token)
#             message=client.messages.create(
#             body="Fall Detected",
#             from_=keys.twilio_number,
#             to=keys.my_phone_number
#             )
#         elif val == 0:
#             print("no fall detected")

In [9]:
# frame=cv2.imread("test-yolo.jpg")

# pred_vid(frame)

In [10]:
cap = cv2.VideoCapture("http://192.168.108.124:4747/video")
#cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()
while True:
    ret, frame = cap.read()

    if not ret:
        print("Error: Could not read frame.")
        break
    frame = cv2.resize(frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    window_width = 600  
    window_height = 600
    cv2.namedWindow('Webcam', cv2.WINDOW_NORMAL)  
    cv2.resizeWindow('Webcam', window_width, window_height)
    cv2.imshow('Webcam', frame)
    key = cv2.waitKey(1)
    if key == ord('p'):
        values=pred_vid(frame)
        if (len(values)==0):
            print("no person detected")
        else:
            for val in values:
                if val == 1:
                    print("fall detected")
                    client=Client(keys.account_sid,keys.auth_token)
                    message=client.messages.create(
                    body="Fall Detected",
                    from_=keys.twilio_number,
                    to=keys.my_phone_number
                    )
                elif val == 0:
                    print("no fall detected")
            
    elif key == ord('q'): 
        break
cap.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 1 chair, 74.9ms
Speed: 0.0ms preprocess, 74.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Bounding box coordinates: (216.00, 99.51), (329.99, 412.17)
1
1234
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
[[ 1.5383e-07]]
no fall detected

0: 480x640 1 dog, 1 chair, 1 bed, 84.6ms
Speed: 0.0ms preprocess, 84.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
0
no person detected

0: 480x640 2 persons, 1 chair, 1 bed, 1 laptop, 84.6ms
Speed: 0.0ms preprocess, 84.6ms inference, 8.5ms postprocess per image at shape (1, 3, 480, 640)
Bounding box coordinates: (0.00, 0.00), (238.44, 278.55)
Bounding box coordinates: (0.07, 0.09), (243.64, 470.11)
2
1234
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
[[   0.015158]]
1234
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
[[  1.868e-06]]
no fall detected
no fall detected

0: 480x640 1 person, 1 backpack, 1 book, 90.3ms
Speed: 0.0ms preprocess, 90.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)